# Get Attendance Allowance Data

A script to identify the IDs of Attendence Allowance datasets and used these to download Attendance Allowance data from the Stat-Xplore API.

This should serve as an example of how to get additional Stat-Xplore datasets in the future.

## Import required libraries

In [34]:
# Add the stat_xplore_scraper directory to the system path so that stat_xplore_scraper libraries can be imported
import sys
sys.path.append('.\stat_xplore_scraper')

# Import stat_xplore_scraper libraries
import stat_xplore_table
import stat_xplore_schema

import pandas as pd

In [35]:
# Initialise the API key. This is used when sending requests to the Stat-Xplore API
APIKey = '65794a30655841694f694a4b563151694c434a68624763694f694a49557a49314e694a392e65794a7063334d694f694a7a644849756333526c6247786863694973496e4e3159694936496d39696153357a59584a6e623235705147396a63326b7559323875645773694c434a70595851694f6a45314d5451354f5467304e7a5173496d46315a434936496e4e30636935765a47456966512e5f65352d586f344a7356466d4945546d657a42587339386d7a793738456c6e4e7061643948706e6270646f'

# Initialise the headers that are used when sending requests to the Stat-Xplore API
table_headers = {'APIKey':APIKey,
                'Content-Type':'applciation/json'}
schema_headers = {'APIKey':APIKey}

# Set the output directory to save data to
output_directory = '..\..\Data\\'

 ## Get the Stat-Xplore schema and identify the required IDs for Attendance Allowance data

In [36]:
# Get the Stat-Xplore schema. This is used to find the codes of fields and values when getting data
df_schema = stat_xplore_schema.get_full_schema(schema_headers, check_cache = True, schema_filename = '.\stat_xplore_scraper\schema.csv')

In [37]:
df_schema.loc[ df_schema['label'] == 'Attendance Allowance']

,id,label,location,parent_id,type
1,str:folder:faa,Attendance Allowance,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:folder:root,FOLDER


In [38]:
# Attendance Allowance ID is 'str:folder:faa'. Find what is avalable in this folder
df_schema.loc[ df_schema['parent_id'] == 'str:folder:faa']

,id,label,location,parent_id,type
17,str:database:AA_In_Payment,AA Cases In Payment,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:folder:faa,DATABASE
18,str:database:AA_Entitled,AA Entitled,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:folder:faa,DATABASE


In [39]:
# Get the data for AA Entitled.
# This shows both the number of people in receipt of an allowance and those with entitlement where the payment has been suspended, for example if they are in hospital.
database_id = 'str:database:AA_Entitled'

In [40]:
# Find the measures and fileds available within this dataset
df_schema.loc[ df_schema['parent_id'] == database_id]

,id,label,location,parent_id,type
68,str:count:AA_Entitled:V_F_AA_Entitled,AA (entitled),http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:database:AA_Entitled,COUNT
69,str:field:AA_Entitled:F_AA_QTR:DATE_NAME,Quarter,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:database:AA_Entitled,FIELD
70,str:folder:AA_Entitled:Geography+%28residence-...,Geography (residence-based),http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:database:AA_Entitled,FOLDER
71,str:field:AA_Entitled:V_F_AA_Entitled:AAAWD,AA Award Type,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:database:AA_Entitled,FIELD
72,str:field:AA_Entitled:V_F_AA_Entitled:CNAGE,Age (bands and single year),http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:database:AA_Entitled,FIELD
73,str:field:AA_Entitled:V_F_AA_Entitled:CTDURTN,Duration of Current Claim,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:database:AA_Entitled,FIELD
74,str:field:AA_Entitled:V_F_AA_Entitled:CCSEX,Gender,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:database:AA_Entitled,FIELD
75,str:field:AA_Entitled:V_F_AA_Entitled:DISABLED,Main Disabling Condition,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:database:AA_Entitled,FIELD


Now select the IDs of the measure to get data for and the field to break the data down by.
Some important points:
* The meaure id must be of type 'COUNT' or 'MEASURE' (these are the types of ordinal data field)
* The stat_xplore_scraper currently only handles requests for data broken down by THREE fields so three fields must be requested
* Since we are interested in geographic data, the geography field should be included
* There is not currently a simple way to select just the latest timepoint. Therefore the date field should also be included to ensure values are broken down by date.

These caveats eman that there is effictively one free choice of field to break data down by.

In [41]:
# Select the measure id - the id of the measure to view get data for
measure_id = 'str:count:AA_Entitled:V_F_AA_Entitled'

In [42]:
# set the data field ID
date_field_id = 'str:field:AA_Entitled:F_AA_QTR:DATE_NAME'

In [43]:
# I also choose to get data broken down by 'Main Disabling Condition'
condition_field_id = 'str:field:AA_Entitled:V_F_AA_Entitled:DISABLED'

In [44]:
# Select the geography field ID

# First get the ID of the geography folder
geog_folder_id = df_schema.loc[ (df_schema['parent_id'] == database_id) & (df_schema['label'] == 'Geography (residence-based)'), 'id'].values[0]

df_schema.loc[ df_schema['parent_id'] == geog_folder_id]

,id,label,location,parent_id,type
521,str:field:AA_Entitled:V_F_AA_Entitled:COA_CODE,National - Regional - LA - OAs,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:folder:AA_Entitled:Geography+%28residence-...,FIELD
522,str:field:AA_Entitled:V_F_AA_Entitled:PARLC_code,Westminster Parliamentary Constituencies,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:folder:AA_Entitled:Geography+%28residence-...,FIELD
523,str:field:AA_Entitled:V_F_AA_Entitled:SPARLC_code,Scottish Parliamentary Constituencies,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:folder:AA_Entitled:Geography+%28residence-...,FIELD


In [45]:
# We want LA data so select the ID of the 'National - Regional - LA - OAs' field
geog_field_id = 'str:field:AA_Entitled:V_F_AA_Entitled:COA_CODE'

In [46]:
# Get the labels of the geography folder, field and level that we want data returned for
geog_folder = 'Geography (residence-based)'
geog_field = 'National - Regional - LA - OAs'

# Use the schema to view what geography levels are available within the geography field
df_schema.loc[ df_schema['parent_id'] == geog_field_id]

,id,label,location,parent_id,type
1069,str:valueset:AA_Entitled:V_F_AA_Entitled:COA_C...,Census Output Area,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:field:AA_Entitled:V_F_AA_Entitled:COA_CODE,VALUESET
1070,str:valueset:AA_Entitled:V_F_AA_Entitled:COA_C...,Lower Layer Super Output Areas,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:field:AA_Entitled:V_F_AA_Entitled:COA_CODE,VALUESET
1071,str:valueset:AA_Entitled:V_F_AA_Entitled:COA_C...,Middle Layer Super Output Areas,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:field:AA_Entitled:V_F_AA_Entitled:COA_CODE,VALUESET
1072,str:valueset:AA_Entitled:V_F_AA_Entitled:COA_C...,Local Authority,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:field:AA_Entitled:V_F_AA_Entitled:COA_CODE,VALUESET
1073,str:valueset:AA_Entitled:V_F_AA_Entitled:COA_C...,Region,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:field:AA_Entitled:V_F_AA_Entitled:COA_CODE,VALUESET
1074,str:valueset:AA_Entitled:V_F_AA_Entitled:COA_C...,Country,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:field:AA_Entitled:V_F_AA_Entitled:COA_CODE,VALUESET
1075,str:valueset:AA_Entitled:V_F_AA_Entitled:COA_C...,Great Britain,http://stat-xplore.dwp.gov.uk/webapi/rest/v1/s...,str:field:AA_Entitled:V_F_AA_Entitled:COA_CODE,VALUESET


In [47]:
# Select the desired geography level. Selecting 'Local Authority' returns data for all local authorities
geog_level = 'Local Authority'

## With the IDs and labels retrieved from the schema, data can now be requested

In [51]:
# Get Attendance Allowance data by calling the stat_xplore_table.get_stat_xplore_measure_data() function
# Selects the fields for which the total number across all values should be included
aa = stat_xplore_table.get_stat_xplore_measure_data(table_headers, 
                                                    schema_headers, 
                                                    measure_id = measure_id, 
                                                    field_ids = [date_field_id,condition_field_id,geog_field_id],
                                                    fields_include_total = condition_field_id, 
                                                    df_schema = df_schema, 
                                                    geog_folder_label = geog_folder,
                                                    geog_field_label = geog_field,
                                                    geog_level_label = geog_level)

In [54]:
# get_stat_xplore_measure_data() returns a dict with keys 'data' and 'annotations'
# save both of these
aa['data'].to_csv(output_directory + 'attendance_allowance_data.csv', index=False)
with open(output_directory + 'aa_annotations.txt', 'w') as f:
    for annotation in aa['annotations'].values():
        f.write(annotation+'\n\n')
